In [36]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [37]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
kaufman_sets = [
'c85de504-96cf-4856-b7f3-398e6bf1e37b',
'c50006d4-a98d-4366-bd5b-650fee1696ee',
'43586c61-a527-43d3-b95a-1bc063fa1fb7',
'e2b336fb-dda9-4eff-9125-0603cc99916e',
'f41f9b4b-9f13-4e25-abed-13fcf99b5a73',
'1ed849f5-cf49-4190-84a9-161f13906c21',
'c33e4a8d-cd5e-4bf2-8973-88a46efd2173',
'd5fd5f93-8ffc-4bdc-8db9-1457d119248b',
]
sprite_sets = ['cc82a2af-95e7-4447-b571-87d4785b33ec']

track_loop_sets = [
'9b1063f5-c891-44a8-b44a-3b4879257366',
'c5db8085-92bd-4d06-a22c-7958e4a78e19',]

my_exp = track_loop_sets[1]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




c5db8085-92bd-4d06-a22c-7958e4a78e19
workflow 2324ad76-ff37-4157-8bcc-3ce72b7dace9 released to project
workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
172
172


In [38]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        for raw_data in store[a_type]:
            print '    SKIPPING FILES', a_type, raw_data['status'], raw_data['uuid']
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            #ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab c5db8085-92bd-4d06-a22c-7958e4a78e19
quality_metric_fastqc in review by lab eb3bd116-f4c3-472e-b465-50f6ca843733
quality_metric_fastqc in review by lab 5a5e490b-05cb-4046-9918-54dc2f9c6a3e
quality_metric_fastqc in review by lab 84f2ae0b-ac9f-4bea-8664-c78de1b4d42d
quality_metric_fastqc in review by lab 878c1d9d-d79c-4d42-af66-bb5f69220b9f
quality_metric_fastqc in review by lab 2537b792-159c-48ce-844f-ca64e81eb808
quality_metric_fastqc in review by lab eee2f4a5-adef-4584-827d-f8fc94372218
quality_metric_fastqc in review by lab 080c5063-228f-402f-8d24-569f31af7b00
quality_metric_fastqc in review by lab 7dcf0489-6734-47c3-a042-09066611a8a5
quality_metric_fastqc in review by lab 43e9d4e8-50c2-427e-b125-4b1cd4206ffc
quality_metric_fastqc in review by lab 40c4f2a8-b76e-4d3e-a35c-b4c30b282265
quality_metric_fastqc in review by lab c9f14cbb-0b56-44f8-99e2-327799a73220
quality_metric_fastqc in review by lab b06bf038-3e1f-4e5c-b2fd-95a7d937d5e7
quality_m

In [39]:
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "submission in progress"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab c5db8085-92bd-4d06-a22c-7958e4a78e19
quality_metric_fastqc in review by lab eb3bd116-f4c3-472e-b465-50f6ca843733
quality_metric_fastqc in review by lab 5a5e490b-05cb-4046-9918-54dc2f9c6a3e
quality_metric_fastqc in review by lab 84f2ae0b-ac9f-4bea-8664-c78de1b4d42d
quality_metric_fastqc in review by lab 878c1d9d-d79c-4d42-af66-bb5f69220b9f
quality_metric_fastqc in review by lab 2537b792-159c-48ce-844f-ca64e81eb808
quality_metric_fastqc in review by lab eee2f4a5-adef-4584-827d-f8fc94372218
quality_metric_fastqc in review by lab 080c5063-228f-402f-8d24-569f31af7b00
quality_metric_fastqc in review by lab 7dcf0489-6734-47c3-a042-09066611a8a5
quality_metric_fastqc in review by lab 43e9d4e8-50c2-427e-b125-4b1cd4206ffc
quality_metric_fastqc in review by lab 40c4f2a8-b76e-4d3e-a35c-b4c30b282265
quality_metric_fastqc in review by lab c9f14cbb-0b56-44f8-99e2-327799a73220
quality_metric_fastqc in review by lab b06bf038-3e1f-4e5c-b2fd-95a7d937d5e7
quality_m